In [4]:
# Backprop on the Seeds Dataset
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            floats = [float(x) for x in row]
            if not row:
                continue
            dataset.append(floats)
    return dataset

# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        #for i in range(len(row)-1):
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

# Transfer neuron activation
def transfer(activation):
    #return 1.0 / (1.0 + exp(-activation))
    return np.tanh(activation)

# Calculate the derivative of an neuron output
def transfer_derivative(output):
    #return output * (1.0 - output)
    return 1.0 - np.tanh(output)**2

# Forward propagate feito para lidar com quaisquer camadas ocultas, camada de saída é linear
def forward_propagate3(network, row):
    inputs = row
    for layer in network[:-1]:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    output_layer = network[-1]
    new_outputs = []
    for output_neuron in output_layer:
        activation = activate(output_neuron['weights'], inputs)
        output_neuron['output'] = activation
        new_outputs.append(output_neuron['output'])
    outputs = new_outputs
    return outputs

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                error = expected[j] - neuron['output']
                errors.append(error)
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

# Treinamento com expected setado com base em valores float
def train_network2(network, dataset, l_rate, n_epoch, n_outputs, stop_rate):
    expected = list()
    sum_error_old = 0.000
    for i in range(len(dataset)-1):
        expected.append(dataset[i+1][-1])
    for epoch in range(n_epoch):
        sum_error = 0
        for i in range(len(dataset)-1):
            valor = dataset[i][0]
            outputs = forward_propagate3(network, dataset[i])
            expected_value = expected[i]
            sum_error += (expected_value-sum(outputs))**2
            backward_propagate_error(network, expected)
            update_weights(network, dataset[i], l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
        if (sum_error_old > 0 and abs(sum_error_old-sum_error) < stop_rate): 
            print("O sum error old é %f, e o sum error atual é %f" %(sum_error_old, sum_error))
            print("SAI DO MEU TREINAMENTO, CONVERGIU!")
            break
        sum_error_old = sum_error
        
# Treinamento com recorrência
def train_network(network, dataset, l_rate, n_epoch, n_outputs, stop_rate, atrasos):
    expected = list()
    sum_error_old = 0.000
    lista_inputs = list()
    for i in range(len(dataset)-1):
        expected.append(dataset[i+1][-1])
    for epoch in range(n_epoch):
        sum_error = 0
        for i in range (0, atrasos+1):
            lista_inputs.append(0)
        ## ACRESCENTA OS INPUTS A CADA ITERAÇÃO DE ATRASO
        for i in range(0, atrasos):
            lista_inputs.insert(1, dataset[i][0])
            lista_inputs.pop(0)
            inputs = lista_inputs
            outputs = forward_propagate3(network, inputs)
            expected_value = expected[i]
            sum_error += (expected_value-sum(outputs))**2  
            backward_propagate_error(network, expected)
            update_weights(network, dataset[i], l_rate) 
            lista_inputs.append(sum(outputs))
            
        ## APÓS OS ATRASOS FEITOS, COMEÇA A PROCESSAR O RESTO DA LISTA
        for i in range(atrasos, len(dataset)-1):
            lista_inputs.insert(1, dataset[i][0])
            # retira a entrada anterior da lista de inputs
            lista_inputs.pop(0)
            inputs = lista_inputs
            outputs = forward_propagate3(network, inputs)
            expected_value = expected[i]
            sum_error += (expected_value-sum(outputs))**2
            backward_propagate_error(network, expected)
            update_weights(network, dataset[i], l_rate)
            lista_inputs.append(sum(outputs))
            # retira o elemento atrasado mais velho da lista de inputs
            lista_inputs.pop(1)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
        # repopula a lista
        lista_inputs.clear()
        if (sum_error_old > 0 and abs(sum_error_old-sum_error) < stop_rate): 
            return sum_error
        sum_error_old = sum_error
    return sum_error

# Initialize a network - tradicional, com bias no output
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

# Initialize a network - tradicional, com atraso
def initialize_network2(n_inputs, n_hidden, n_outputs, n_atrasos):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + n_atrasos + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

# Initialize a network - tradicional, com atraso, arquitetura com várias hidden layers
def initialize_network3(n_inputs, n_hidden, n_layers, n_outputs, n_atrasos):
    network = list()
    for i in range(n_layers):
        hidden_layer = [{'weights':[random() for i in range(n_inputs + n_atrasos + 1)]} for i in range(n_hidden)]
        network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

# Acrescenta mais um valor na camada de entrada e a seu correspondente na camada oculta
def acrescenta_input(network):
    camada = network[0]
    neuronio = camada[0]
    network.reverse()
    neuronio['weights'].append(random())
    network.reverse()
    return network

# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return sum(outputs)

#Soma os valores de uma saída e faz a reconversão Min-Max
def calcula_saida(layer):
    cont = 0
    #return sum(layer)
    for i in range(len(layer)):
        neuron = layer[i]
        cont += neuron['output']
    #cont = transfer(cont)
    #return (maprange((0,1),(-1,1),cont))
    return cont
    #return scaler.inverse_transform(cont)
    
# converte range de saída
def maprange(a, b, s):
    (a1, a2), (b1, b2) = a, b
    return  b1 + ((s - a1) * (b2 - b1) / (a2 - a1))

def testa_rede (treinamento, n_inputs, n_outputs, n_hidden, n_atrasos, n_epoch, stop_rate):
    melhor = 100
    hidden = 0
    atrasos = 0
    for i in range(1, n_hidden):
        for j in range(0, n_atrasos):
            rede = initialize_network2(n_inputs,n_hidden,n_outputs,n_atrasos)
            x = train_network(rede, treinamento, l_rate, n_epoch, n_inputs, stop_rate, n_atrasos)
            if (x < melhor):
                melhor = x
                hidden = n_hidden
                atrasos = n_atrasos
    print("A melhor rede foi a com %d hidden e %d atrasos, tendo um resultado de %f de erro" %(hidden,atrasos,melhor))
    
    
    
# Test Backprop on Seeds dataset
seed(1)
# load and prepare data
filename = 'serie2.csv'
treinamento = load_csv(filename)
#scaler = MinMaxScaler(feature_range=(-1, 1))
#dataset = scaler.fit_transform(treinamento)
# normalize input variables
minmax = dataset_minmax(treinamento)
normalize_dataset(treinamento, minmax)

# evaluate algorithm
l_rate = 0.3
n_epoch = 1000
n_hidden = 10
n_inputs = len(treinamento[0])
n_outputs = 1
stop_rate = 10e-5
n_atrasos = 10
n_layers = 2
network = initialize_network2(n_inputs, n_hidden, n_outputs, n_atrasos)
#network = initialize_network3(n_inputs, n_hidden, n_layers, n_outputs, n_atrasos)
train_network(network, treinamento, l_rate, n_epoch, n_inputs, stop_rate, n_atrasos)
#testa_rede(treinamento,n_inputs,n_outputs,n_hidden,n_atrasos,n_epoch, stop_rate)

FileNotFoundError: [Errno 2] No such file or directory: 'serie2.csv'